In [1]:
import pandas as pd
import numpy as np
from scipy import stats

def check_un_demographics(file_path, year=2023):
    # 1. Load and Clean (Matching prepare_dataviz.py)
    df = pd.read_csv(file_path, low_memory=False)
    
    # Columns mapping (Key in JS -> Column in CSV)
    cols = {
        'fertilityRate': 'Total Fertility Rate (live births per woman)',
        'medianAge': 'Median Age, as of 1 July (years)',
        'infantMortality': 'Infant Mortality Rate (infant deaths per 1,000 live births)',
        'lifeExpectancy': 'Life Expectancy at Birth, both sexes (years)',
        'migrationRate': 'Net Migration Rate (per 1,000 population)',
        'birthRate': 'Crude Birth Rate (births per 1,000 population)'
    }
    
    for col in cols.values():
        if df[col].dtype == 'object':
            df[col] = df[col].astype(str).str.strip().str.replace(' ', '', regex=False)
        df[col] = pd.to_numeric(df[col], errors='coerce')
    
    # Filter for Countries only in the specified year
    data = df[(df['Type'] == 'Country/Area') & (df['Year'] == year)].copy()
    
    def get_regression_stats(x_key, y_key):
        # Subset and drop NaNs
        subset = data[[cols[x_key], cols[y_key]]].dropna()
        subset.columns = ['x', 'y']
        
        # 2. IQR Outlier Removal (Matching statistics.js)
        def filter_iqr(d, col):
            q1, q3 = d[col].quantile([0.25, 0.75])
            iqr = q3 - q1
            return d[(d[col] >= q1 - 1.5 * iqr) & (d[col] <= q3 * 1.5 + iqr)] # Adjusted for 1.5 * IQR

        # Apply to both X and Y
        clean = filter_iqr(subset, 'x')
        clean = filter_iqr(clean, 'y')
        
        # 3. Regression Analysis
        slope, intercept, r_val, p_val, std_err = stats.linregress(clean['x'], clean['y'])
        return {
            "n": len(clean),
            "r2": r_val**2,
            "slope": slope,
            "removed": len(subset) - len(clean)
        }

    # Execute tests
    results = {
        "Fertility vs Median Age": get_regression_stats('fertilityRate', 'medianAge'),
        "Infant Mortality vs Life Expectancy": get_regression_stats('infantMortality', 'lifeExpectancy'),
        "Migration vs Birth Rate": get_regression_stats('migrationRate', 'birthRate')
    }
    
    for test, res in results.items():
        print(f"--- {test} ---")
        print(f"Sample Size: {res['n']} | R²: {res['r2']:.4f} | Slope: {res['slope']:.4f} | Outliers Removed: {res['removed']}")

# Run the check
check_un_demographics('data/world-demographic.csv')

--- Fertility vs Median Age ---
Sample Size: 232 | R²: 0.7259 | Slope: -7.6907 | Outliers Removed: 5
--- Infant Mortality vs Life Expectancy ---
Sample Size: 226 | R²: 0.7539 | Slope: -0.4188 | Outliers Removed: 11
--- Migration vs Birth Rate ---
Sample Size: 194 | R²: 0.0318 | Slope: -0.5172 | Outliers Removed: 43


In [3]:
import pandas as pd
from scipy import stats

def discover_insights(file_path, year=2023):
    df = pd.read_csv(file_path, low_memory=False)
    # Filter for Countries only
    data = df[(df['Type'] == 'Country/Area') & (df['Year'] == year)].copy()
    
    # List of key analytical indicators
    indicators = [
        "Median Age, as of 1 July (years)",
        "Crude Birth Rate (births per 1,000 population)",
        "Total Fertility Rate (live births per woman)",
        "Crude Death Rate (deaths per 1,000 population)",
        "Life Expectancy at Birth, both sexes (years)",
        "Infant Mortality Rate (infant deaths per 1,000 live births)",
        "Net Migration Rate (per 1,000 population)",
        "Population Growth Rate (percentage)"
    ]

    # Pre-clean: numeric conversion
    for col in indicators:
        data[col] = pd.to_numeric(data[col].astype(str).str.replace(' ', ''), errors='coerce')

    print(f"--- Automated Demographic Discovery for {year} ---")
    results = []
    for i, x_col in enumerate(indicators):
        for y_col in indicators[i+1:]:
            subset = data[[x_col, y_col]].dropna()
            if len(subset) < 30: continue
            
            # IQR Outlier Removal (Sync with your dashboard logic)
            def filter_iqr(d, c):
                q1, q3 = d[c].quantile([0.25, 0.75])
                iqr = q3 - q1
                return d[(d[c] >= q1 - 1.5*iqr) & (d[c] <= q3 + 1.5*iqr)]
            
            clean = filter_iqr(filter_iqr(subset, x_col), y_col)
            slope, intercept, r_val, p_val, _ = stats.linregress(clean[x_col], clean[y_col])
            results.append({'x': x_col, 'y': y_col, 'r2': r_val**2, 'slope': slope})

    # Sort and display top correlations
    for res in sorted(results, key=lambda x: x['r2'], reverse=True)[:5]:
        print(f"Discovery: {res['x']} vs {res['y']}")
        print(f"Result: R² = {res['r2']:.3f} | Slope: {res['slope']:.3f}\n")

discover_insights('data/world-demographic.csv')

--- Automated Demographic Discovery for 2023 ---
Discovery: Crude Birth Rate (births per 1,000 population) vs Total Fertility Rate (live births per woman)
Result: R² = 0.954 | Slope: 0.117

Discovery: Median Age, as of 1 July (years) vs Crude Birth Rate (births per 1,000 population)
Result: R² = 0.847 | Slope: -0.883

Discovery: Life Expectancy at Birth, both sexes (years) vs Infant Mortality Rate (infant deaths per 1,000 live births)
Result: R² = 0.743 | Slope: -1.733

Discovery: Median Age, as of 1 July (years) vs Total Fertility Rate (live births per woman)
Result: R² = 0.726 | Slope: -0.092

Discovery: Crude Birth Rate (births per 1,000 population) vs Life Expectancy at Birth, both sexes (years)
Result: R² = 0.698 | Slope: -0.622

